### Notebook for Webscraping Data to get information about German Startups

In [20]:
# import requests
from pip._vendor import requests
# making a request to a webpage
page = requests.get("https://www.failory.com/startups/germany", timeout=15)
print(page)
print(page.status_code) ## prints 200 as status code
#print(page.content[:5000]) ## prints the entire webpage content

<Response [200]>
200


In [21]:
from bs4 import BeautifulSoup
import re # a library to do regular expression matching
import pandas as pd
import numpy as np

In [22]:
soup = BeautifulSoup(page.content, "html.parser") # converts the page content into a beautifulsoup object
type(soup)

bs4.BeautifulSoup

In [23]:
soup.find('link').get('href')

'https://global-uploads.webflow.com/5fada95bd7a18db677087e20/css/failory-v3.webflow.7138e7e6a.min.css'

In [24]:
soup.findAll('link')[3].get('href')

'https://www.failory.com/startups/germany'

In [25]:
soup.get_text()[:500]

"300 Best Startups in Germany\n\n\n\nLearn how to validate your startup idea by pre-selling it, for only $80 (includes a free 1-hour consultancy call).FailoryInterviewsBlogGraveyardToolsDealsResourcesDownload The List of the\xa0100 Highest-Valued UnicornsThis free sheet contains all the information about the top 100 unicorns, including their valuation, HQ's location, founded year, name of founders, funding amount and number of employees.\n\nGet the Free Sheet\nDownload The List of the\xa0100 Highest-Valued Pr"

In [26]:
company_title = soup.findAll('h3')
#company_title.partition(')')[0]
#company_title

In [27]:
soup.findAll('ul',{'role': 'list'})[0]

<ul role="list"><li>City: Munich</li><li>Started in: 2011</li><li>Founders: Alexander Rinke, Bastian Nominacher, Martin Klenk</li><li>Industries: Analytics, Business Intelligence, SaaS, Software</li><li>Number of employees: 1001-5000</li><li>Funding amount: $1,367,500,000</li><li>Number of funding rounds: 4</li><li>Number of investors: 9</li></ul>

In [28]:
len(soup.findAll('ul',{'role': 'list'}))

300

In [29]:
soup.findAll('ul',{'role': 'list'})[0].attrs

{'role': 'list'}

In [30]:
info = soup.findAll('ul',{'role': 'list'})[1].findAll('li')
len(info)
info

[<li>City: Berlin</li>,
 <li>Number of employees: 51-100</li>,
 <li>Funding amount: $87,201,448</li>]

In [31]:
[i.get_text().strip() for i in info]

['City: Berlin', 'Number of employees: 51-100', 'Funding amount: $87,201,448']

In [32]:
info

[<li>City: Berlin</li>,
 <li>Number of employees: 51-100</li>,
 <li>Funding amount: $87,201,448</li>]

In [33]:
info_used = ["City: ","Started in: ","Founders: ","Industries: ","Number of employees: ","Funding amount: ",
            "Number of funding rounds: ","Number of investors: "]
#info_used = []
for i in info:
    if(i.get_text().strip().partition(':')[0] == "City"):
        info_used[0] = i.get_text().strip()
    elif(i.get_text().strip().partition(':')[0] == "Number of employees"):
        info_used[4] = i.get_text().strip()    
##info_used.append(i.get_text().strip().partition(':')[2])
#info_used

In [34]:
info_used = []
for i in info:
    info_used.append(i.get_text().strip())
info_used

['City: Berlin', 'Number of employees: 51-100', 'Funding amount: $87,201,448']

In [35]:
start_date = info[1].text

In [36]:
start_date.partition(':')[2]

' 51-100'

In [37]:
founders = info[2].text.partition(':')[2]
founders

' $87,201,448'

In [38]:
founders.partition(':')[2]

''

In [39]:
city = info[0].text.split()

In [40]:
city[1]

'Berlin'

In [41]:
company_name = [] 
for i in company_title:
    company_name.append(i.get_text().strip())    

In [42]:
print(company_name[1].partition(')')[2])

 TradeRepublic


In [43]:
test_company = company_name[0].split()
test_company[1]

'Celonis'

In [44]:
info_used = soup.findAll('ul',{'role': 'list'})[1].findAll('li')
info_used

[<li>City: Berlin</li>,
 <li>Number of employees: 51-100</li>,
 <li>Funding amount: $87,201,448</li>]

In [45]:
cols = ["Company", "City", "Started_in", "Founders", "Industries", "Number_of_employees",
            "Funding_amount", "Number_of_funding_rounds", "Number_of_investors"]
    
df = pd.DataFrame(columns = cols)

#for i in range(len(soup.findAll('ul',{'role': 'list'}))):
for i in range(2):

    info = ["City: ","Started in: ","Founders: ","Industries: ","Number of employees: ","Funding amount: ",
            "Number of funding rounds: ","Number of investors: "]
    info_used = soup.findAll('ul',{'role': 'list'})[i].findAll('li')
    for j in info_used:
        if(j.get_text().strip().partition(':')[0] == "City"):
            info[0] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Started in"):
            info[1] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Founders"):
            info[2] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Industries"):
            info[3] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Number of employees"):
            info[4] = j.get_text().strip()                
        elif(j.get_text().strip().partition(':')[0] == "Funding amount"):
            info[5] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Number of funding rounds"):
            info[6] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Number of investors"):
            info[7] = j.get_text().strip()
            
    try:
        City = info[0].text.partition(':')[2]
    except AttributeError:
        City = np.nan
    
    #print(city)
    print(info[0].partition(':')[2])

 Munich
 Berlin


In [46]:
cols = ["Company", "City", "Started_in", "Founders", "Industries", "Number_of_employees",
            "Funding_amount", "Number_of_funding_rounds", "Number_of_investors"]
    
df = pd.DataFrame(columns = cols)

for i in range(len(soup.findAll('ul',{'role': 'list'}))):
    
    info = ["City: ","Started in: ","Founders: ","Industries: ","Number of employees: ","Funding amount: ",
            "Number of funding rounds: ","Number of investors: "]
    info_used = soup.findAll('ul',{'role': 'list'})[i].findAll('li')

    #info_used
    for j in info_used:
        if(j.get_text().strip().partition(':')[0] == "City"):
            info[0] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Started in"):
            info[1] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Founders"):
            info[2] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Industries"):
            info[3] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Number of employees"):
            info[4] = j.get_text().strip()                
        elif(j.get_text().strip().partition(':')[0] == "Funding amount"):
            info[5] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Number of funding rounds"):
            info[6] = j.get_text().strip()
        elif(j.get_text().strip().partition(':')[0] == "Number of investors"):
            info[7] = j.get_text().strip()
     
        
    try:
        test_company = company_name[i].split()
        Company = test_company[1]
    except AttributeError:
        Company = np.nan

    try:
        City = info[0].partition(':')[2]
    except AttributeError:
        City = np.nan

    try:
        Started_in = info[1].partition(':')[2]
    except AttributeError:
        Started_in = np.nan
            
    try:
        Founders =  info[2].partition(':')[2]
    except AttributeError:
        Founders = np.nan
            
    try:
        Industries = info[3].partition(':')[2]
    except AttributeError:
        Industries = np.nan

    try:
        Number_of_employees = info[4].partition(':')[2]
    except AttributeError:
        Number_of_employees = np.nan

    try:
        Funding_amount = info[5].partition(':')[2]
    except AttributeError:
        Funding_amount = np.nan

    try:
        Number_of_funding_rounds = info[6].partition(':')[2]
    except AttributeError:
        Number_of_funding_rounds = np.nan
            
    try:
        Number_of_investors = info[7].partition(':')[2]
    except AttributeError:
        Number_of_investors = np.nan
            
    job_dict = { 
          'Company': Company, 
          'City': City, 
          'Started_in': Started_in, 
          'Founders': Founders,
          'Industries': Industries,
          'Number_of_employees': Number_of_employees, 
          'Funding_amount': Funding_amount,
          'Number_of_funding_rounds': Number_of_funding_rounds,  
          'Number_of_investors': Number_of_investors }
    #print(job_dict)
    df = df.append(job_dict, ignore_index=True)    


C:\Users\Luismi\AppData\Local\Temp\ipykernel_19724\4001696001.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(job_dict, ignore_index=True)
C:\Users\Luismi\AppData\Local\Temp\ipykernel_19724\4001696001.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(job_dict, ignore_index=True)
C:\Users\Luismi\AppData\Local\Temp\ipykernel_19724\4001696001.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(job_dict, ignore_index=True)
C:\Users\Luismi\AppData\Local\Temp\ipykernel_19724\4001696001.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(job_dict, ignore_index=True)
C:\Users

In [50]:
df

,Company,City,Started_in,Founders,Industries,Number_of_employees,Funding_amount,Number_of_funding_rounds,Number_of_investors
0,Celonis,Munich,2011,"Alexander Rinke, Bastian Nominacher, Martin K...","Analytics, Business Intelligence, SaaS, Software",1001-5000,"$1,367,500,000",4,9
1,TradeRepublic,Berlin,,,,51-100,"$87,201,448",,
2,Ada,Berlin,2011,"Claire Novorol, Daniel Nathrath, Martin Hirsch","Apps, Artificial Intelligence, Health Care, M...",101-250,"$159,497,394",4,10
3,Delivery,Berlin,,,,10001,"$5,144,433,803",,
4,uberall,Berlin,2012,"David Federhen, Florian Hubner, Josha Benner","Information Technology, Location Based Servic...",101-250,"$170,800,000",8,7
...,...,...,...,...,...,...,...,...,...
295,Finanzcheck,Hamburg,,,,251-500,"$53,267,667",,
296,Layer,Berlin,2019,"Constantin Schünemann, David Barnowsky, Ernes...",Software,1-10,"€ 5,700,000",2,7
297,E-Farm.com,Hamburg,,,,11-50,"$5,300,000",,
298,XAIN,Berlin,,,,11-50,"$7,035,853",,
